In [1]:
import pandas as pd
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec

In [3]:
df = pd.read_csv('data/Form relationships.csv')
df = df[['COLUMN_NAME', 'TABLE_NAME', 'ORDINAL_POSITION']]
df = df.applymap(str)
odf = df

# Grouping one from -> one row of dataframe
gdf = df.groupby('TABLE_NAME')

Split COLUMN_NAME

In [4]:
# Split by UpperCase
def splitAtUpperCase(s):
    for i in range(len(s)-1)[::-1]:
        if s[i].isupper() and s[i+1].islower():
            s = s[:i]+' '+s[i:]
        if s[i].isupper() and s[i-1].islower():
            s = s[:i]+' '+s[i:]
            
    return s[1:]  #s.split()

#e.g.
splitAtUpperCase('TheLongANDWindingRoad')

'CDCustomer'
'Long AND Winding'

'Long AND Winding'

In [5]:
splitAtUpperCase('ATOnumber')

'AT Onumber'

In [1]:
from polyglot.text import Text

blob = "TheLongANDWindingRoad."
text = Text(blob)
text.language = "en"
print(text.morphemes)

ModuleNotFoundError: No module named 'icu'

In [10]:
s = "ATOnumber"

for i in range(len(s)-1)[::-1]:
    
    if s[i].isupper() and s[i+1].islower():
        print(s[i])
        s = s[:i]+' '+s[i:]
        
        
    if s[i].isupper() and s[i-1].islower():
        s = s[:i]+' '+s[i:]

s[1:]  #s.split()  


O


'ATO number'

In [ ]:
for index, row in df.iterrows():
    row['COLUMN_NAME'] = splitAtUpperCase(row['COLUMN_NAME'])
    
df.head()

,COLUMN_NAME,TABLE_NAME,ORDINAL_POSITION
0,Data Protection Key Id,__CDDataProtectionKeys,1
1,Friendly Name,__CDDataProtectionKeys,2
2,Xml,__CDDataProtectionKeys,3
3,System Name,__CDDataProtectionKeys,4
4,Migration Id,__MigrationHistory,1


In [ ]:
#read dummy table

ddf = pd.read_csv('Form relationships_dummy tables.csv')
ddf = ddf[['COLUMN_NAME', 'TABLE_NAME', 'ORDINAL_POSITION']]

for index, row in ddf.iterrows():
    row['COLUMN_NAME'] = splitAtUpperCase(row['COLUMN_NAME'])
    
ddf = ddf.applymap(str)
gdf = ddf.groupby('TABLE_NAME')

Processing dataset

In [ ]:
# Dataframe to one paragraph

def df_to_sentence(df):
    lst = df.values.tolist()

    sentence = ''
    for row in lst:
        for el in row:
            if el != 'nan':
                sentence = sentence + el + ' '
    return sentence


In [ ]:
# iterate form -> sentence -> concatenat into a list [,,]
train_data = []

for table_name, df in gdf:
    sentence = df_to_sentence(df)
    train_data.append({table_name: sentence})

train_data[:2]

[{'A': 'OntologyAuditId A 1 OntologyEventId A 2 Action A 3 ActionDate A 4 ActionUserProfileId A 5 Field A 6 FieldName A 7 PreviousValue A 8 NewValue A 9 OntologyId A 10 '},
 {'A+B': 'OntologyAuditId A+B 1 OntologyEventId A+B 2 Action A+B 3 ActionDate A+B 4 ActionUserProfileId A+B 5 Field A+B 6 FieldName A+B 7 PreviousValue A+B 8 NewValue A+B 9 OntologyId A+B 10 OntologyEntityAuditId A+B 11 OntologyEntityEventId A+B 12 Action A+B 13 ActionDate A+B 14 ActionUserProfileId A+B 15 Field A+B 16 FieldName A+B 17 PreviousValue A+B 18 NewValue A+B 19 OntologyEntityId A+B 20 '}]

In [ ]:
# Tokenizing / Tagging Data
# Don't need to stringify -> tokenize it again because we have tokenized dataset already


#todo: get rid of table name from train data. Add once only.

tagged_data_train = []

for i, _d in enumerate(train_data):
    for x in _d:
        tagged_data_train.append(TaggedDocument(words=word_tokenize(_d[x]), tags=[x]))

tagged_data_train[:2]


[TaggedDocument(words=['OntologyAuditId', 'A', '1', 'OntologyEventId', 'A', '2', 'Action', 'A', '3', 'ActionDate', 'A', '4', 'ActionUserProfileId', 'A', '5', 'Field', 'A', '6', 'FieldName', 'A', '7', 'PreviousValue', 'A', '8', 'NewValue', 'A', '9', 'OntologyId', 'A', '10'], tags=['A']),
 TaggedDocument(words=['OntologyAuditId', 'A+B', '1', 'OntologyEventId', 'A+B', '2', 'Action', 'A+B', '3', 'ActionDate', 'A+B', '4', 'ActionUserProfileId', 'A+B', '5', 'Field', 'A+B', '6', 'FieldName', 'A+B', '7', 'PreviousValue', 'A+B', '8', 'NewValue', 'A+B', '9', 'OntologyId', 'A+B', '10', 'OntologyEntityAuditId', 'A+B', '11', 'OntologyEntityEventId', 'A+B', '12', 'Action', 'A+B', '13', 'ActionDate', 'A+B', '14', 'ActionUserProfileId', 'A+B', '15', 'Field', 'A+B', '16', 'FieldName', 'A+B', '17', 'PreviousValue', 'A+B', '18', 'NewValue', 'A+B', '19', 'OntologyEntityId', 'A+B', '20'], tags=['A+B'])]

Testing SyllableTokenizer 
https://www.nltk.org/api/nltk.tokenize.html#submodules

In [ ]:
import nltk

words = word_tokenize('Agent, ... Action Audit Id AgentActionAudit 1 Agent Action Event Id AgentActionAudit 2 Action AgentActionAudit 3 Action Date AgentActionAudit 4 Action User Profile Id AgentActionAudit 5 Field AgentActionAudit 6 Field Name AgentActionAudit 7 Previous Value AgentActionAudit 8 New Value AgentActionAudit 9 Agent Action Id AgentActionAudit 10 ')
print(words)

#nltk.tokenize.SyllableTokenizer


['Agent', ',', '...', 'Action', 'Audit', 'Id', 'AgentActionAudit', '1', 'Agent', 'Action', 'Event', 'Id', 'AgentActionAudit', '2', 'Action', 'AgentActionAudit', '3', 'Action', 'Date', 'AgentActionAudit', '4', 'Action', 'User', 'Profile', 'Id', 'AgentActionAudit', '5', 'Field', 'AgentActionAudit', '6', 'Field', 'Name', 'AgentActionAudit', '7', 'Previous', 'Value', 'AgentActionAudit', '8', 'New', 'Value', 'AgentActionAudit', '9', 'Agent', 'Action', 'Id', 'AgentActionAudit', '10']


Train w/ Train data

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=500)
model.build_vocab(tagged_data_train)
model.train(tagged_data_train, total_examples=model.corpus_count, epochs=500)
model.save("d2v.model")

Result

In [ ]:
# Pulling similar form ranking of form 'OntologyAudit'
similar_doc = model.docvecs.most_similar('OntologyAudit')
similar_doc

[('OntologyStyleAudit', 0.798603892326355),
 ('OntologyEntityStyleAudit', 0.783116340637207),
 ('OntologyEvent', 0.7270788550376892),
 ('OntologyEntityAudit', 0.7136366367340088),
 ('OntologyEntityInstanceAudit', 0.6952971816062927),
 ('DBPediaOntologyImportAudit', 0.6896032094955444),
 ('OntologyAttachment', 0.6817100644111633),
 ('OntologyStyleEvent', 0.6809934973716736),
 ('OntologyEntityEvent', 0.6511623859405518),
 ('Ontology', 0.6460370421409607)]

In [ ]:
# Pulling similar form ranking of form 'UserProfile'
similar_doc = model.docvecs.most_similar('UserProfile')
similar_doc

[('UserPreference', 0.7616549730300903),
 ('HTMLDocumentEvent', 0.6327312588691711),
 ('Role', 0.6066591739654541),
 ('HTMLDocumentAudit', 0.5961733460426331),
 ('RoleEvent', 0.5834612250328064),
 ('RoleAudit', 0.5753050446510315),
 ('TextDocument', 0.5360458493232727),
 ('HTMLDocumentAttachment', 0.5340317487716675),
 ('ScheduledTask', 0.5227214097976685),
 ('TextDocumentAudit', 0.515395998954773)]

Load saved model

In [ ]:
loaded_model = Doc2Vec.load("d2v.model")

Pairwise matrix

In [ ]:
gdf = odf.groupby('TABLE_NAME')

tables = pd.DataFrame(gdf).loc[:,0]

In [ ]:
print (loaded_model.docvecs.similarity('UserProfile', 'UserPreference'))
print (loaded_model.docvecs.distance('UserProfile', 'UserPreference'))

0.7616549
0.23834508657455444


In [ ]:
    #discussion about unseen_docs
    
    
    
    # Gensim code https://tedboy.github.io/nlps/_modules/gensim/models/doc2vec.html
    
    # def similarity(self, d1, d2):
    #     """
    #     Compute cosine similarity between two docvecs in the trained set, specified by int index or
    #     string tag. (TODO: Accept vectors of out-of-training-set docs, as if from inference.)
    #     """
    #     return dot(matutils.unitvec(self[d1]), matutils.unitvec(self[d2]))
    
    # def similarity_unseen_docs(self, model, doc_words1, doc_words2, alpha=0.1, min_alpha=0.0001, steps=5):
        # """
        # Compute cosine similarity between two post-bulk out of training documents.

        # Document should be a list of (word) tokens.
        # """
        # d1 = model.infer_vector(doc_words=doc_words1, alpha=alpha, min_alpha=min_alpha, steps=steps)
        # d2 = model.infer_vector(doc_words=doc_words2, alpha=alpha, min_alpha=min_alpha, steps=steps)
        # return dot(matutils.unitvec(d1), matutils.unitvec(d2))

In [ ]:
# pairwise matrix testing for dummy table
gdf = ddf.groupby('TABLE_NAME')

tables = pd.DataFrame(gdf).loc[:,0]
tables

0      A
1    A+B
2     A1
3     A2
4     A3
5     A4
6      B
7     B1
8      C
9      D
Name: 0, dtype: object

In [ ]:
# table x table score matrix
lst = []

for x in tables:
    score_x = []
    for y in tables:
        if x == y:
            score_x.append(0) # res dist is not exactly 0. Set to 0 here.
        else:
            score_x.append(loaded_model.docvecs.distance(x,y)) 
            #options 1)add to traindata 2)infervector & calculate distance
            
            # A : [1,2,3,,3,3,3......]
            # B : []
            
    lst.append(score_x)
    

KeyError: "tag 'A' not seen in training corpus/invalid"

In [ ]:
# list to df
matrix = pd.DataFrame.from_records(lst)
matrix.set_axis(tables, axis=1, inplace=True)
matrix.rename(tables, inplace=True)
matrix.head()

,AgentAction,AgentActionAudit,AgentActionEvent,AgentActionImplementation,AgentActionImplementationAudit,AgentActionImplementationEvent,AgentObservable,AgentObservableAudit,AgentObservableEvent,AgentVersion,...,TreeVariableDeclaration,TreeVariableDeclarationAudit,TreeVariableDeclarationEvent,TreeVariableEvent,UserPreference,UserProfile,UserProfileAudit,UserProfileEvent,__CDDataProtectionKeys,__MigrationHistory
AgentAction,0.000000,0.426657,0.360168,0.231590,0.351109,0.390691,0.216480,0.441973,0.467782,0.284959,...,0.672596,0.607704,0.666957,0.705735,0.781006,0.935765,0.786522,0.926871,0.481608,0.384993
AgentActionAudit,0.426657,0.000000,0.291344,0.279186,0.222073,0.363804,0.301075,0.309346,0.393429,0.284061,...,0.769332,0.442447,0.700367,0.787564,0.619720,0.770890,0.666344,0.795262,0.714500,0.399622
AgentActionEvent,0.360168,0.291344,0.000000,0.272522,0.296338,0.108977,0.362235,0.325286,0.145750,0.370165,...,0.891696,0.673737,0.591577,0.661197,0.734881,0.927735,0.736268,0.579181,0.977538,0.581733
AgentActionImplementation,0.231590,0.279186,0.272522,0.000000,0.288982,0.287368,0.235428,0.403516,0.413050,0.296297,...,0.870687,0.659856,0.703113,0.761652,0.701897,0.865002,0.882472,0.786362,0.598558,0.374147
AgentActionImplementationAudit,0.351109,0.222073,0.296338,0.288982,0.000000,0.186103,0.348016,0.264830,0.397451,0.445815,...,0.777886,0.532420,0.800159,0.861916,0.691898,0.828001,0.519832,0.686927,0.744446,0.610098


In [ ]:
# df to csv
matrix.to_csv('result_matrix.csv')

Distance Tree

In [ ]:
from skbio import DistanceMatrix
from skbio.tree import nj
#run: pip install scikit-bio

In [ ]:
dm = DistanceMatrix(matrix, tables)
tree = nj(dm)
str_tree = tree.ascii_art()
print(str_tree)

                                                            /-SMATAStateAttachment
                                                  /--------|
                                                 |          \-EpisodeStateAttachment
                                        /--------|
                                       |         |          /-OntologyAttachment
                                       |          \--------|
                              /--------|                    \-DBPediaOntologyImportAttachment
                             |         |
                             |         |          /-MLModelParameterAttachment
                             |          \--------|
                             |                   |          /-MLWipAttachment
                             |                    \--------|
                             |                              \-MLDataSetAttachment
                             |
                             |                              /

In [ ]:
text_file = open("tree.txt", "w")
text_file.write(str_tree)
text_file.close()
